In [1]:
import os 
import pandas as pd
from bokeh.plotting import figure, show 
from bokeh.layouts import gridplot
from ipywidgets import interact, interact_manual

In [2]:

from bokeh.io import output_notebook
output_notebook()
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import HoverTool
from math import pi

Loading BokehJS ...

In [3]:
#this is the path to the chicago-police-data GitHub, 
#used to access unified_data.zip and discipline_penalty_codes.csv
#data_path = '/Users/Sarah/Documents/GitHub/chicago-police-data/data'

#Code has been changed so that you can access all of the data from one location
#instences of data_path have been changed to path
#all of the needed data is in my HW5 Repository



#this is the save-to path, this is where the unified_data.zip unzips to
#and where we read the accused, investigators and victims data from (out of the unzipped contents)
#path =  '/Users/Sarah/Documents/GitHub/assignment-4-gillsarah'
path= '/Users/Sarah/Documents/GitHub/assignment-5-sarah-gill-1'

#profile_path = 'unified_data/profiles/officer-profiles.csv.gz' #path for reading from chicago-police-data
profile_path = 'fully-unified-data/profiles/officer-profiles.csv.gz'
codes_path = 'context_data/discipline_penalty_codes.csv'
base_path = 'fully-unified-data/complaints/complaints-{}_2000-2016_2016-11.csv.gz'
file_name = ['accused', 'investigators', 'victims']


def pathmaker(base_path, file):
    return base_path.format(file)


def unzip(path, filename, save_to_path):
    zf = ZipFile(os.path.join(path, filename), 'r')
    zf.extractall(save_to_path)
    zf.close()
    #cite: https://stackoverflow.com/questions/3451111/unzipping-files-in-python


def read_df(path, filename):
    df = pd.read_csv(os.path.join(path, filename))
    return df


def parse_accused(accused_df):
    accused_df.drop(columns = 'row_id', inplace = True)
    final_dummies = pd.get_dummies(accused_df['final_finding'])
    #recommend_dummies = pd.get_dummies(accused_df['recommended_finding'])
    #cite https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
    accused_df['sustained'] = final_dummies['SU']
    #accused_df['recommend_sustain'] = recommend_dummies['SU']
    return accused_df


def parse_investigarots(investigators_df):
    investigators_df.drop(columns = 'row_id', inplace = True)
    investigators_df.rename(columns = {'first_name':'investigator_first_name', 
                                       'last_name':'investigator_last_name',
                                       'middle_initial':'investigator_middle_initial', 
                                       'suffix_name':'investigator_suffix',
                                       'appointed_date': 'date_investigator_appointed', 
                                       'current_star':'investigator_current_star_number',
                                       'current_rank': 'investigator_current_rank', 
                                       'current_unit':'investigator_current_unit'}, inplace = True)
    return investigators_df

def parse_victims(victims_df):
    victims_df.rename(columns = {'gender':'victim_gender', 'age':'victim_age', 
                                 'race':'victim_race'}, inplace = True)
    return victims_df

def parse_profile(profile_df):
    profile_df['org_hire_date'] = pd.to_datetime(profile_df['org_hire_date'], 
                                                 format='%Y-%m-%d')
    #profile_df['birth_year'] = pd.to_datetime(profile_df['birth_year'], format='%Y')
    profile_df['Year_hired'] = profile_df['org_hire_date'].map(lambda d: d.year)
    return profile_df

def merge_dfs(dfs):
    '''
    takes a list of dfs, the order is decided in the list dfs. If you change the order, the function
    may need to be tweeked. The suffixes, and merges 3 and 4
    The frist df is accused, the second is investigators, the third is victims, the third is codes
    '''
    merge_0 = dfs[0].merge(dfs[4], how = 'left', on = 'UID', suffixes = ('_accused', '_profile'))
    merge_1 = merge_0.merge(dfs[1], how = 'inner', on =  "cr_id", suffixes = ('_accused','_investigators'))
    merge_2 = merge_1.merge(dfs[2], how = 'inner', on =  "cr_id")
    merge_3 = merge_2.merge(dfs[3], how = 'left', left_on = 'recommended_discipline', right_on = 'CODE')

    merge_3.drop(columns = 'CODE', inplace = True)
    merge_3.rename(columns={'recommended_discipline': 'recommended_discipline_code',
                            'ACTION_TAKEN'          : 'recommended_discipline'      }, inplace = True)       
    
    merge_4 = merge_3.merge(dfs[3], how = 'left', left_on = 'final_discipline', 
                            right_on = 'CODE',suffixes = ('_recommended_discipline', '_final_discipline'))

    merge_4.drop(columns = 'CODE', inplace = True)

    merge_4.rename(columns={'final_discipline': 'final_discipline_code',
                            'ACTION_TAKEN'    : 'final_discipline'     }, inplace = True) 
    merge_4['count'] = 1 
    return merge_4


#proportion sustained: looking at all complaints filed, one entry per accused individual
def total_proportion(accused_df):
    return accused_df['sustained'].sum()/len(accused_df.index)

#proportion sustained of complaints that have a line in victims, investigarots and accused 
def outcome_by_race(df, outcome_word):
    grouped = df.groupby('victim_race').sum()
    grouped[outcome_word]
    grouped['proportion_'+outcome_word] = grouped[outcome_word]/len(df.index)
    df = grouped['proportion_'+outcome_word]

    #print('The proportion of complaints '+outcome_word+', by race of the victim:')
    #print(grouped['proportion_'+outcome_word])
    return df


def complaint_type_outcomes(accused_df, outcome, outcome_word):
    '''
    takes the acccused df, the two letter string for final finding: e.g. 'SU' and a string 
    for the final finding abbreviation meaning (e.g. 'sustained' for 'SU')
    output is a list of the complaint catagories for which the outcome (e.g. 'SU')
    is the most likely final finding
    '''
    crosstab = pd.crosstab(accused_df['final_finding'], accused_df['complaint_category'])
    #cite https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html

    #print('The following complaint catagories are most likely to be ' + outcome_word + ':')
    temp_list = []
    for column in crosstab.columns:
        if crosstab[column].idxmax() == outcome:
            print(column)
            temp_list.append(column)
    #cite: https://stackoverflow.com/questions/15741759/find-maximum-value-of-a-column-and-return-the-corresponding-row-values-using-pan
    df = pd.DataFrame(temp_list, columns = ['complaint catagories most likely to be ' + outcome_word]) 
    #cite: https://www.geeksforgeeks.org/create-a-pandas-dataframe-from-lists/
    return df

def export_df(df, path, filename):
    df.to_csv(os.path.join(path, filename))



def main():
    files = []
    for f in file_name:
        files.append(pathmaker(base_path, f))

    #commented out for useing only one data path
    #unzip(data_path, 'unified_data/unified_data.zip', path)

    dfs = []
    for filename in files:
        df = read_df(path, filename)
        if filename.__contains__('accused'):
            dfs.append(parse_accused(df))
        elif filename.__contains__('investigators'):
            dfs.append(parse_investigarots(df))
        elif filename.__contains__('victims'):
            dfs.append(parse_victims(df))
        else:
            print('unexpected file')
    dfs.append(read_df(path, codes_path))
    
    df2 = read_df(path, profile_path)
    dfs.append(parse_profile(df2))

    df = merge_dfs(dfs)

    #proportion = total_proportion(dfs[0])
    #print('Total proportion of complaints that are sustained: {:.4f}'.format(proportion))
    #print(' ')

    #race_df = outcome_by_race(df, 'sustained')


    #outcome_df = complaint_type_outcomes(dfs[0], 'SU', 'sustained')

    export_df(df, path, 'full_df.csv')
    #export_df(race_df, path, 'Proportion of compliants sustained by race.csv')
    #export_df(outcome_df, path, 'Most likely to be sustained.csv')

    return df


df = main()

/Users/Sarah/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
def filter_df(df, col1, col2='gender', value1 = 'all', value2= 'all'):
    if value1 == 'all':
        filtered_df = df
    else:
        filtered_df = df[df[col1] == value1]
    
    if value2 == 'all':
        filtered_df2 = filtered_df
    else:
        filtered_df2 = filtered_df[filtered_df[col2]== value2]
    return filtered_df2

In [4]:
def small_df_maker(df, col1, col2, col3 = 'count'):
    '''
    takes a df and two strings: 3 column names in that df, that you want to work with
    returns a dictionary ready to be used in to my_fn
    '''
    
    drop_list = []
    for colname in df.columns:
        if colname in [col1, col2, col3]:
            pass
        else:
            drop_list.append(colname)
    df2 = df.drop(columns = drop_list)
    df3 = dict(tuple(df2.groupby(col1)))
    #cite https://stackoverflow.com/questions/19790790/splitting-dataframe-into-multiple-dataframes
    return df3



In [23]:
def my_fn(df, officer_string, col1, col2, col3 = 'count', title = " "):
    '''
    takes a string value for officer, what you want to groubby
    e.g. 'WHITE' or 'MALE'
    this needs to line up with the values for col1 (e.g. 'race' or 'gender')
    takes a string value for the colum name 
    e.g. 'victim_race' or 'victim_gender'
    output is a bar graph of col2 grouped by col1
    '''
    df2 = small_df_maker(df, col1, col2, col3 = 'count')
    g = df2[officer_string].groupby(col2, as_index = False).sum()
    #g.reset_index(inplace=True)
    
    hover = HoverTool(tooltips = [('count', '@top')])
    #cite https://towardsdatascience.com/data-visualization-with-bokeh-in-python-part-one-getting-started-a11655a467d4
       
    plot = figure(x_range= g[col2], plot_height=450, plot_width = 1500, title=title,
           toolbar_location=None, tools=[hover])

    plot.xaxis.major_label_orientation = pi/4
    #cite https://stackoverflow.com/questions/42354648/how-to-rotate-x-axis-labels-in-bokeh-figure
    #cite https://docs.bokeh.org/en/latest/docs/user_guide/styling.html#tick-label-orientation
    plot.vbar(g[col2], top = g[col3], width = 0.8)
    #cite https://docs.bokeh.org/en/latest/docs/user_guide/categorical.html
    
    return plot

In [6]:
officer_race = df.reset_index()['race'].unique()
#officer_race

In [ ]:
@interact(race_of_officer=officer_race) #dropdown menue
def make_plot_for(race_of_officer= officer_race[0]):
    tabs = tab(df, race_of_officer, 'race', 'victim_race', 'final_finding', 'cleaned_rank',
               'Victim Race', 'Complaint Finding', 'Officer Rank') 
    show(tabs) #show the plot

    #rate: complaints against this type of officer: % by this grouping 
    #can make the x-axis on an angle

In [7]:
df['complaint_category'].fillna('not listed', inplace = True)
#cite: https://stackoverflow.com/questions/13295735/how-can-i-replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-datafra

In [16]:
complaint_cat = df.reset_index()['complaint_category'].unique()
#complaint_cat
race = df.reset_index()['race'].unique()

In [24]:
#@interact_manual(type_of_complaint=complaint_cat, sustained_only=False) #dropdown menue
@interact_manual(race=race, sustained_only=False)
#def make_plot_for(type_of_complaint= complaint_cat[0], sustained_only=False):
def make_plot_for(race = race[0], sustained_only=False):
    if sustained_only:
        #df_new = filter_df(df, 'race', value1='SU')
        df_new = filter_df(df, 'complaint_category', value1='SU')
        
    else:
        df_new = df
     #type_of_complaint   
    plot = my_fn(df_new, race , col2= 'complaint_category', col1 = 'race', 
                 title = "Final Discipline by Type of Complaint" )
    show(plot)

interactive(children=(Dropdown(description='race', options=('WHITE', 'HISPANIC', 'BLACK', 'ASIAN/PACIFIC ISLAN…